# Investment Opportunities and Alignment

- how investments consistent with the Paris Agreement Long-term target look like
- the investments of today determine to a large extent the emissions of tomorrow

In [126]:
using CSV
using Statistics
using JSON

## Helper functions

In [193]:
function averageYears(row, years)
    # We loop over the years and get the value for each year from the row
    # Since the column name of the years are in Symbols we need to convert the String-year
    # From this new array we calculate the mean and cound it with 3 digits
    function getYear(year)
        value = row[1, Symbol(year)]
        if (typeof(value) === String) # In some cases we need to convert the data from the csv to a number
           parse(Float64, replace(value, "," => ".")) 
        else
            value
        end
    end
    round(mean(getYear, years), digits=3)
end

averageYears (generic function with 1 method)

## Getting the historic data

In [128]:
historicData = CSV.read("source/WEI2020-DataTables_supplytimeseries_BNEF.csv", decimal='.')

,Model,Scenario,Region,Variable
,String,String,String,String
1,Reference,IEA-WEI,World,Investment|Energy Supply|Extraction|Coal
2,Reference,IEA-WEI,World,Investment|Energy Supply|Extraction|Fossil
3,Reference,IEA-WEI,World,Investment|Energy Supply|Electricity|Nuclear
4,Reference,IEA-WEI,World,Investment|Energy Supply|Electricity|Fossil
5,Reference,IEA-WEI,World,Investment|Energy Supply|Electricity|Transmission and Distribution
6,Reference,BNEF,World,Investment|Energy Supply|Electricity|Solar
7,Reference,BNEF,World,Investment|Energy Supply|Electricity|Wind
8,Reference,IEA-WEI,World,Investment|Energy Efficiency
9,Reference,IEA-WEI,World,Investment|Energy Supply|Extraction|Biomass


### Coresponding key names

In [129]:
VARIABLE_MAPPING = Dict{String,String}(
    "Energy Efficiency" => "Investment|Energy Efficiency",
    "Electricity - T&D and Storage" => "Investment|Energy Supply|Electricity|Transmission and Distribution",
    "Extraction and Conversion - Nuclear" => "Investment|Energy Supply|Electricity|Nuclear",
    "Extraction and Conversion - Bioenergy" => "Investment|Energy Supply|Extraction|Biomass",
    "Energy Supply|Electricity|Solar" => "Investment|Energy Supply|Electricity|Solar",
    "Energy Supply|Electricity|Wind" => "Investment|Energy Supply|Electricity|Wind",
    "Electricity - Fossil Fuels w/o CCS" => "Investment|Energy Supply|Electricity|Fossil",
    "Extraction and Conversion - Fossil Fuels" => "Investment|Energy Supply|Extraction|Fossil",
    "Coal" => "Investment|Energy Supply|Extraction|Coal"
)

Dict{String,String} with 9 entries:
  "Energy Efficiency"       => "Investment|Energy Efficiency"
  "Extraction and Conversi… => "Investment|Energy Supply|Extraction|Fossil"
  "Extraction and Conversi… => "Investment|Energy Supply|Extraction|Biomass"
  "Coal"                    => "Investment|Energy Supply|Extraction|Coal"
  "Extraction and Conversi… => "Investment|Energy Supply|Electricity|Nuclear"
  "Electricity - Fossil Fu… => "Investment|Energy Supply|Electricity|Fossil"
  "Energy Supply|Electrici… => "Investment|Energy Supply|Electricity|Solar"
  "Energy Supply|Electrici… => "Investment|Energy Supply|Electricity|Wind"
  "Electricity - T&D and S… => "Investment|Energy Supply|Electricity|Transmissi…

### Relevant historic years

In [130]:
HISTORIC_YEARS = (2017, 2018, 2019)

(2017, 2018, 2019)

In [131]:
function findHistoricData(variable)
    row = filter(row -> row[:Variable] == variable, historicData)
    averageYears(row, HISTORIC_YEARS)
end

findHistoricData (generic function with 1 method)

In [132]:
# Test function
begin
    findHistoricData("Investment|Energy Efficiency")
end

213.765

In [188]:
function getHistoricData(variable)
    dict = Dict{String, Float64}()
    dict["Reference"] = 0 # Some variable are not present in the historic data so we use 0 for that.
    
    if (haskey(VARIABLE_MAPPING, variable)) # Some variables have different names in the historic data so we map the keys to the corresponding ones.
        dict["Reference"] = findHistoricData(VARIABLE_MAPPING[variable])
    elseif (variable === "Oil and Gas") # Some variables need to be specificly calculated
        fossil = findHistoricData("Investment|Energy Supply|Extraction|Fossil")
        coal = findHistoricData("Investment|Energy Supply|Extraction|Coal")
        dict["Reference"] = fossil - coal
    end
    
    # println(variable, "->", dict["Reference"])
    dict["Reference"] = round(dict["Reference"], digits=3)
    dict["average"] = dict["Reference"] # Since we don’t have any models for the historic data, the average is the same
    dict["max"] = dict["Reference"] # The same applies for the maximum value
    return dict
end

getHistoricData (generic function with 1 method)

In [134]:
scenarioRuns = CSV.read("source/41560_2018_179_MOESM2_ESM-1.csv", decimal=',')

,Model,Region,Scenario,Variable
,String,String,String,String
1,AIM/CGE,World,CPol,Total energy investment
2,AIM/CGE,World,CPol,Total energy investment (supply side)
3,AIM/CGE,World,CPol,Low carbon investment
4,AIM/CGE,World,CPol,Low carbon investment (supply side)
5,AIM/CGE,World,CPol,Total_inv/GDP
6,AIM/CGE,World,CPol,LC_inv/GDP
7,AIM/CGE,World,CPol,Extraction and Conversion - Fossil Fuels
8,AIM/CGE,World,CPol,Electricity - Fossil Fuels w/o CCS
9,AIM/CGE,World,CPol,Hydrogen - Fossil


In [135]:
unique(scenarioRuns[!,:Variable])

31-element Array{String,1}:
 "Total energy investment"                                
 "Total energy investment (supply side)"                  
 "Low carbon investment"                                  
 "Low carbon investment (supply side)"                    
 "Total_inv/GDP"                                          
 "LC_inv/GDP"                                             
 "Extraction and Conversion - Fossil Fuels"               
 "Electricity - Fossil Fuels w/o CCS"                     
 "Hydrogen - Fossil"                                      
 "Electricity - Non-bio Renewables"                       
 "Hydrogen - Non-fossil"                                  
 "Extraction and Conversion - Bioenergy"                  
 "Extraction and Conversion - Nuclear"                    
 ⋮                                                        
 "Energy Supply|Electricity|Gas|w/o CCS"                  
 "Energy Supply|Electricity|Oil|w/ CCS"                   
 "Energy Supply|Electricity|

## Defining valid values

In [136]:
VARIABLES = (
  "Energy Efficiency",
  "CCS",
  "Electricity - T&D and Storage",
  "Extraction and Conversion - Nuclear",
  "Extraction and Conversion - Bioenergy",
  "Hydrogen - Non-fossil",
  "Electricity - Non-bio Renewables", # This will only be used to calculate other values. See getValues function
  "Energy Supply|Electricity|Solar",
  "Energy Supply|Electricity|Wind",
  "Hydrogen - Fossil",
  "Electricity - Fossil Fuels w/o CCS",
  "Extraction and Conversion - Fossil Fuels", # This will only be used to calculate other values. See getValues function
  "other renewables", # This is not in the data and will be calculated by "Electricity - Non-bio Renewables" minus "Energy Supply|Electricity|Solar" and "Energy Supply|Electricity|Wind". See getValues function
  "Coal", # This is not in the data and will be calculated by "Extraction and Conversion - Fossil Fuels" * 0.1. See getValues function
  "Oil and Gas" # This is not in the data and will be calculated by "Extraction and Conversion - Fossil Fuels" * 0.9. See getValues function
)

("Energy Efficiency", "CCS", "Electricity - T&D and Storage", "Extraction and Conversion - Nuclear", "Extraction and Conversion - Bioenergy", "Hydrogen - Non-fossil", "Electricity - Non-bio Renewables", "Energy Supply|Electricity|Solar", "Energy Supply|Electricity|Wind", "Hydrogen - Fossil", "Electricity - Fossil Fuels w/o CCS", "Extraction and Conversion - Fossil Fuels", "other renewables", "Coal", "Oil and Gas")

In [137]:
MODELS = (
  "AIM/CGE",
  "IMAGE",
  "MESSAGEix-GLOBIOM",
  "POLES",
  "REMIND-MAgPIE"
)

("AIM/CGE", "IMAGE", "MESSAGEix-GLOBIOM", "POLES", "REMIND-MAgPIE")

In [138]:
REGIONS = (
  "World",
  "CHN",
  "IND",
  "USA",
  "R5OECD90+EU",
  "R5REF",
  "R5ASIA",
  "R5MAF",
  "R5LAM",
  "EU"
)

("World", "CHN", "IND", "USA", "R5OECD90+EU", "R5REF", "R5ASIA", "R5MAF", "R5LAM", "EU")

In [139]:
SCENARIOS = (
  "1.5C",
  "NDC",
  "2C",
  "CPol"
)

("1.5C", "NDC", "2C", "CPol")

The `in` function checks if the first parameter is in second parameter.

## Filtering runs

After defining the valid values, we filter the runs by checking if each attributes is present in the value tuples.

In [140]:
function included(d)
    return in(d[:Variable], VARIABLES) && in(d[:Model], MODELS) && in(d[:Region], REGIONS) && in(d[:Scenario], SCENARIOS)
end

datum = filter(included, scenarioRuns)

,Model,Region,Scenario,Variable,Unit
,String,String,String,String,String
1,AIM/CGE,World,CPol,Extraction and Conversion - Fossil Fuels,Billion US$2015/yr
2,AIM/CGE,World,CPol,Electricity - Fossil Fuels w/o CCS,Billion US$2015/yr
3,AIM/CGE,World,CPol,Hydrogen - Fossil,Billion US$2015/yr
4,AIM/CGE,World,CPol,Electricity - Non-bio Renewables,Billion US$2015/yr
5,AIM/CGE,World,CPol,Hydrogen - Non-fossil,Billion US$2015/yr
6,AIM/CGE,World,CPol,Extraction and Conversion - Bioenergy,Billion US$2015/yr
7,AIM/CGE,World,CPol,Extraction and Conversion - Nuclear,Billion US$2015/yr
8,AIM/CGE,World,CPol,Electricity - T&D and Storage,Billion US$2015/yr
9,AIM/CGE,World,CPol,CCS,Billion US$2015/yr


## Prepare functions

### Summing up values

Each year has it own column in the source data and we want to sum it up as one value.
Additionally, we need to parse the value and convert it from German notation to English.

In [141]:
NEAR_TERM_YEARS = (2020, 2025, 2030)
MEDIUM_TERM_YEARS = (2025, 2030, 2035, 2040, 2045, 2050)

(2025, 2030, 2035, 2040, 2045, 2050)

To get a value in a DataFrame we use [x, y], where `x` is the row number and `y` the cell id. This would usually look something like `[1, :Region]`, but in this special case the id of the column is a `Symbol` with the name of the year. So we use `Symbol(year)`.

In [142]:
function sumYears(row)
    sum = 0
    for year in YEARS
        raw = row[1, Symbol(year)]
        value = parse(Float64, replace(raw, "," => "."))
        sum += value
    end
    return round(sum, digits=3)
end

sumYears (generic function with 1 method)

### GetValues
This function finds the rows in the DataFrame, gets every value and calculates the average.

First, we create an empty Dictionary with `Strings` as keys and `Float64` as numbers.
We loop over the `MODELS` and search for each row with the current model.
If we found one, we summarise the years and add the result to the Dictionary

After the loop, we get the values of the dictionary and calculate the mean value.

In [187]:
function getValues(scenario, variable, region)
    dict = Dict{String, Float64}()
    
    for model in MODELS
        # In some cases we need to calculate the values
        if (variable === "other renewables")
            renewables = findScenarioData(scenario, "Electricity - Non-bio Renewables", region, model)
            wind = findScenarioData(scenario, "Energy Supply|Electricity|Wind", region, model)
            solar = findScenarioData(scenario, "Energy Supply|Electricity|Solar", region, model)
            dict[model] = renewables - wind - solar
        elseif (variable === "Coal")
            fossilFuels = findScenarioData(scenario, "Extraction and Conversion - Fossil Fuels", region, model)
            dict[model] = dict[model] = fossilFuels * 0.1
        elseif (variable === "Oil and Gas")
            fossilFuels = findScenarioData(scenario, "Extraction and Conversion - Fossil Fuels", region, model)
            dict[model] = fossilFuels * 0.9
        else # End of the special cases
            dict[model] = findScenarioData(scenario, variable, region, model)
        end
    end
    
    dict["average"] = round(mean(values(dict)), digits=3) # We calculate the average by simply getting the mean of all values
    dict["max"] = maximum(values(dict)) # We calculate the maximum value from all values
    return dict
end

getValues (generic function with 1 method)

### Filter function
The filter function takes two parameter: a function returning true or false for each item; and a list of items.

In [186]:
function findScenarioData(scenario, variable, region, model)
    row = filter(row -> row[:Scenario] == scenario && row[:Region] == region && row[:Variable] == variable && row[:Model] == model, datum)
    if (size(row, 1) > 0)
        return averageYears(row, MEDIUM_TERM_YEARS)
    else
        return 0
    end
end

findScenarioData (generic function with 2 methods)

In [145]:
begin
    findScenarioData("NDC", "Extraction and Conversion - Nuclear", "World", "AIM/CGE")
end

NDC


99.884

### Calculating the change

Function to calculate the change between two values.
We calculate the absolute change and if its positive or negative. This makes it easier to process later.

In [146]:
function calcChange(reference, value)
    if (reference === value)
       return (0, true) 
    end
    ref = reference === 0.0 ? 1 : reference # In order to prevent the value from being Infinity, we change the value to 1 if it is 0
    change = round(abs(ref - value) / ref, digits = 3)
    # change = round(abs(reference - value) / max(reference, value), digits=3)
    isPositive = ref >= value
    # if (reference === 0.0)
    #    println("reference: ", reference, " value:", value, " change: ", change)
    # end
    return (min(200, change), isPositive) # We limit the change to 200 percent
end

calcChange (generic function with 1 method)

### Calculating the changes

This function iterates over all values, gets its refeference value and saves the change in the dictionary.

In [147]:
function calcChanges(vals, refs)
    changes = Dict{String, Any}()
    for (key, val) in vals
        ref = refs["Reference"]
        changes[key] = calcChange(ref, val)
    end
    changes
end

calcChanges (generic function with 1 method)

## Build the final data set

Finally, we loop over all scenarios, variables and regions and calculate everything.

In [191]:
runs = []

for scenario in SCENARIOS
    for variable in VARIABLES
        if (variable !== "Electricity - Non-bio Renewables" && variable !== "Extraction and Conversion - Fossil Fuels")
            for region in REGIONS
                vals = getValues(scenario, variable, region)
                refs = getHistoricData(variable)
                changes = calcChanges(vals, refs)
                push!(runs, (scenario = scenario, variable = variable, region = region, values = vals, reference = refs, changes = changes))
            end
        end
    end
end

## Export the dataset

`JSON.json` takes two arguments: a Dict and the indent.

In [192]:
open("../static/data/investments.json", "w") do io
   write(io, JSON.json(runs, 2));
end;